# Select frontpoints

In [5]:
!pip install geopandas

DEPRECATION: tensorflow model has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of tensorflow or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [6]:
#import the packages needed
import geopandas as gpd
from shapely.geometry import LineString, box
from shapely.ops import split
import matplotlib.pyplot as plt

## Funtion definition 

In [7]:
def select_frontyard_facadeline(points_length, residential_plots, output_path):
    
    #start loading the input files into gdf
     #residential plots
    res_plots = residential_plots
    rplots_gdf = gpd.read_file(res_plots)

    #points that include the lenth to the clossest road
    points_lenth = points_length
    pointslenth_gdb = gpd.read_file(points_lenth)
    
    #create groups that contain the points for each plot
    grouped = pointslenth_gdb.groupby('CAPAKEY')
    
    # Function to select the two points with shortest distances within each group
    def select_two_smallest(group):
        return group.nsmallest(2, 'NEAR_DIST')
    
    # Select the two points with the smallest distances within each group
    points_smallest = grouped.apply(select_two_smallest).reset_index(drop=True)
    
    
    #the following part wil connect these 2 points to create the 'front'-line of the house
    # Group points by 'group' column (CAPAKEY)
    grouped_points = points_smallest.groupby('CAPAKEY')

    # Create an empty list to store LineString geometries
    lines_list = []
    
    # Iterate over each group
    for group_name, group_data in grouped_points:
        # If there's only one point in the group, skip it (can't create a line with just one point)
        if len(group_data) < 2:
            continue

        # Create a LineString geometry from the points in the group
        line = LineString(group_data.geometry.tolist())

        # Append the LineString to the list
        lines_list.append({'CAPAKEY': group_name, 'geometry': line})
    
    # Create a new GeoDataFrame from the list of LineString geometries
    lines = gpd.GeoDataFrame(lines_list, geometry='geometry')
    
    
    #the next part will extend the lines, first overextend and then cut it to the edges of the shape
    #Function to extend a line in both directions
    def extend_line(line, extension_distance):
        # Get the start and end points of the line
        start_point = line.coords[0]
        end_point = line.coords[-1]

        # Calculate the direction vector of the line
        dx = end_point[0] - start_point[0]
        dy = end_point[1] - start_point[1]

        # Extend the line in both directions
        extended_start = (start_point[0] - extension_distance * dx, start_point[1] - extension_distance * dy)
        extended_end = (end_point[0] + extension_distance * dx, end_point[1] + extension_distance * dy)

        # Create the extended line
        extended_line = LineString([extended_start, extended_end])

        return extended_line
    
    # Extension distance
    extension_distance = 1000  # Adjust as needed

    # Create an empty list to store LineString geometries
    extended_lines_list = []

    # Iterate over each line
    for index, row in lines.iterrows():
        line = row['geometry']
        capakey = row['CAPAKEY'] #extract capakey from original

        # Extend the line
        extended_line = extend_line(line, extension_distance)

        # Append the extended line to the list
        extended_lines_list.append({'CAPAKEY': capakey, 'geometry': extended_line})

    # Create a new GeoDataFrame from the list of extended LineString geometries
    extended_lines = gpd.GeoDataFrame(extended_lines_list, geometry='geometry')
    
    # Create an empty list to store the cut LineString geometries
    cut_lines_list = []
    
    # Iterate over each line to cut it down
    for index, row in extended_lines.iterrows():
        # Get the CAPAKEY of the line
        line = row['geometry']
        capakey = row['CAPAKEY']

        # Get the corresponding polygon
        polygon = rplots_gdf[rplots_gdf['CAPAKEY'] == capakey].geometry.iloc[0]

        # Perform intersection between the line and the polygon
        intersection = line.intersection(polygon)

        # Check if the intersection result is not empty and is a LineString or MultiLineString
        if not intersection.is_empty and (
                intersection.geom_type == 'LineString' or intersection.geom_type == 'MultiLineString'):
            # Append the valid intersection to the list of cut lines
            cut_lines_list.append({'CAPAKEY': capakey, 'geometry': intersection})
    
    # Create a new GeoDataFrame from the list of cut LineString geometries
    cut_lines = gpd.GeoDataFrame(cut_lines_list, geometry='geometry')

    # Set the CRS
    cut_lines.set_crs(epsg=31370, inplace=True)

    # Export to a shapefile
    cut_lines.to_file(output_path, driver='ESRI Shapefile')
    
    """
    # Plot the first layer (layer1)
    ax = rplots_gdf.plot(color='blue', edgecolor='black', alpha=0.5, label='Layer 1')

    # Plot the second layer (layer2) on the same axes
    points_smallest.plot(ax=ax, color='red', edgecolor='black', alpha=0.5, label='Layer 2')

    #plit the thirt layer on the same axis
    cut_lines.plot(ax=ax, color='green', edgecolor='black', alpha=0.5, label='Layer 3')

    # Customize the plot (add title, legend, etc.)
    plt.legend()
    # Show the plot
    plt.show()
    """
    
    return print('Finished')

## Execute funtion

In [8]:
filepath_1= r"J:\SET-SADL_User-DI0222\Projecten\H-USAGE\Usecases\Models_EVB\data\facade_points_studyarea_distance\facade_points_studyarea.shp"
filepath_2= r'J:\SET-SADL_User-DI0222\Projecten\H-USAGE\Usecases\Models_EVB\data\residential_plots\residential_plots_municip.shp'
filepath_3= r'J:\SET-SADL_User-DI0222\Projecten\H-USAGE\Usecases\Models_EVB\data\front_lines_buildings\cut_frontlines.shp'
select_frontyard_facadeline(filepath_1, filepath_2, filepath_3)

Finished
